## Import Library

In [9]:
import zipfile
from bs4 import BeautifulSoup
import unicodedata
import re
import pandas as pd
import numpy as np

## Open and read docx

In [10]:
#open docx with zipfile
doc_zip = zipfile.ZipFile('FORM A.docx')
# doc_zip.namelist()

In [11]:
#read xml
doc_xml = doc_zip.read('word/document.xml')

In [12]:
# parse xml 
soup_xml = BeautifulSoup(doc_xml, 'xml')
# find body
body = soup_xml.find('body')

In [13]:
# print all text
for i, p in enumerate(body.find_all(['w:p','w:br'])):
    string = unicodedata.normalize('NFKD', p.get_text())
    # print(i,p.get_text())

In [14]:
len(body.find_all(['w:p','w:br']))

216

## Extract Questions

In [15]:
# construct empty dictionary 'questions'
questions = {'question_line_nbr':[],
      'question':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())

    # if first 4 charactors in the paragraph has 'number dot space' save it to questions dict.
    if bool(re.search('\d+\. ', string[:4])):
        # print(i,string)
        questions['question_line_nbr'].append(i)
        questions['question'].append(string)

In [16]:
# find the last line number and append to the questions dict.
last_line_nbr = len(body.find_all(['w:p','w:br']))-1
questions['question_line_nbr'].append(last_line_nbr)
questions['question'].append(np.nan)

In [17]:
# convert to Dataframe
df_q = pd.DataFrame(questions)

## Extract Answers

### Get Answer A, B, C, and D,  and save them to Dictionary Answers 1, 2, 3, and 4

In [18]:
# construct empty dictionary 'answers1'
answers1 = {'answer1_line_nbr':[],
      'answer1':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())
    
    # if first 4 charactors in the paragraph has 'A dot space' save it to answers1 dict.
    if bool(re.search('A\. ', string[:4])):
        # print(i,string)
        answers1['answer1_line_nbr'].append(i)
        answers1['answer1'].append(string)

In [19]:
# construct empty dictionary 'answers2'
answers2 = {'answer2_line_nbr':[],
      'answer2':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())
    
    # if first 4 charactors in the paragraph has 'B dot space' save it to answers2 dict.
    if bool(re.search('B\. ', string[:4])):
        # print(i,string)
        answers2['answer2_line_nbr'].append(i)
        answers2['answer2'].append(string)

In [20]:
# construct empty dictionary 'answers3'
answers3 = {'answer3_line_nbr':[],
      'answer3':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())
    
    # if first 4 charactors in the paragraph has 'C dot space' save it to answers3 dict.
    if bool(re.search('C\. ', string[:4])):
        # print(i,string)
        answers3['answer3_line_nbr'].append(i)
        answers3['answer3'].append(string)

In [21]:
# construct empty dictionary 'answers4'
answers4 = {'answer4_line_nbr':[],
      'answer4':[]}

for i, p in enumerate(body.find_all('w:p')):
    string = unicodedata.normalize('NFKD', p.get_text())
    
    # if first 4 charactors in the paragraph has 'D dot space' save it to answers4 dict.
    if bool(re.search('D\. ', string[:4])):
        # print(i,string)
        answers4['answer4_line_nbr'].append(i)
        answers4['answer4'].append(string)

In [22]:
# convert to Dataframe
df_a1 = pd.DataFrame(answers1)
df_a2 = pd.DataFrame(answers2)
df_a3 = pd.DataFrame(answers3)
df_a4 = pd.DataFrame(answers4)

## Combine Questions and Answers

In [23]:
# find line number of answer between line number of questions
for i in range(len(df_q)-1):
    start=df_q['question_line_nbr'][i]
    end=df_q['question_line_nbr'][i+1]
    # return pd series containing answer1, 2, 3, and 4 (A, B, C, D) for question 
    answer1 = df_a1[df_a1['answer1_line_nbr'].between(start,end)]['answer1']
    answer2 = df_a2[df_a2['answer2_line_nbr'].between(start,end)]['answer2']
    answer3 = df_a3[df_a3['answer3_line_nbr'].between(start,end)]['answer3']
    answer4 = df_a4[df_a4['answer4_line_nbr'].between(start,end)]['answer4']
    
    # if there is answer for the question append to answer column
    if answer1.empty==False:
        df_q.loc[i,'answer1']=answer1.tolist()[0]
    if answer2.empty==False:
        df_q.loc[i,'answer2']=answer2.tolist()[0]
    if answer3.empty==False:
        df_q.loc[i,'answer3']=answer3.tolist()[0]
    if answer4.empty==False:
        df_q.loc[i,'answer4']=answer4.tolist()[0]

In [24]:
 df_q

,question_line_nbr,question,answer1,answer2,answer3,answer4
0,20,1. A list of four physical quantities is,A. 1,B. 2,C. 3,D. 4
1,31,2. The uncertainty in reading a laboratory th...,A. 2.5 %,B. 5 %,C. 7.5 %,D. 10 %
2,36,3. An object of mass m is hung from a spring. ...,NaN,NaN,NaN,NaN
3,42,"4. Two different experiments, P and Q, generat...",NaN,NaN,NaN,NaN
4,47,5. The road from city X to city Y is 1000 km ...,NaN,NaN,NaN,NaN
5,52,6. An object is thrown upwards leaving the th...,NaN,NaN,NaN,NaN
6,54,7. A car accelerates uniformly from rest to a...,A. v t2,B. 12vt2-t1+v t1,C. 12vt2+t1,D. 12v t1+vt2-t1
7,60,8. The graph below shows how velocity v varie...,A. displacement.,B. final velocity.,C. change in velocity.,D. acceleration.
8,68,9. A boy throws a ball horizontally at a spee...,A. 40 m,B. 60 m,C. 80 m,D. 320 m
9,75,10. A rocket has just been launched vertically...,NaN,NaN,NaN,NaN
